# Collect weather and load data 

This notebook provides all the code and documentation to collect the Ausgrid load dataset and the Meteostat Weather dataset. 


# Australia Emission Data

The fuel mix and emission intensity data is only available for limited times and resolutions.
- https://opennem.org.au/energy/nsw1/?range=all&interval=1M&view=discrete-time

Therefore, we consider monthly emissions and scale them by daily profiles. 

# Ausgrid Load Data

The load datasets can be downloaded manually from the Ausgrid Website. You also find further documentation there. 
- https://www.ausgrid.com.au/Industry/Our-Research/Data-to-share/Solar-home-electricity-data

The half-hour electricity data is for 300 homes with rooftop solar systems that are measured by a gross meter that records the total amount of solar power generated every 30 minutes. The data has been sourced from 300 randomly selected solar customers in Ausgrid’s electricity network area that were billed on a domestic tariff and had a gross metered solar system installed for the whole of the period from 1 July 2010 to 30 June 2013. The customers chosen had a full set of actual data for the period from 1 July 2010 to 30 June 2011, gathered through our meter reading processes. We also undertook some data quality checking and excluded customers on the high and low ends of household consumption and solar generation performance during the first year.

# SMARD Electricity Production Data

The production datasets can be downloaded manually from the SMARD Website:
- https://www.smard.de/home/downloadcenter/download-marktdaten/?downloadAttributes=%7B%22selectedCategory%22:1,%22selectedSubCategory%22:1,%22selectedRegion%22:%22DE%22,%22selectedFileType%22:%22CSV%22,%22from%22:1593554400000,%22to%22:1688162399999%7D

# Australian Energy Market Operator

The electricity price datasets can be downloaded manually from the AEMO Website:

- https://aemo.com.au/energy-systems/electricity/national-electricity-market-nem/data-nem/aggregated-data

# Meteostat Weather Data

The weather data from the Meteostat Homepage can be collected via an API (https://dev.meteostat.net/). Meteostat is an open platform which provides free access to historical weather and climate data.

In [1]:
# Required libraries
import pandas as pd
from datetime import datetime

%pip install meteostat -U
from meteostat import  Daily, Hourly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Set time period for the dataset
start = datetime(2012, 6, 30)
end = datetime(2013, 7, 1)

# Get hourly data
# Provide the needed zip-code
#For more details look at: https://meteostat.net/en/station/94755?t=2023-10-31/2023-11-07
# 94755 -> Zip code for Timezone Australia/ Sydney
hourly_data = Hourly('94755', start, end)

In [3]:
# Get data
df = hourly_data.fetch()
df.head()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2012-06-30 02:00:00,16.0,6.7,54.0,NaN,NaN,350.0,7.6,NaN,1012.6,NaN,NaN
2012-06-30 05:00:00,16.8,4.1,43.0,NaN,NaN,220.0,11.2,NaN,1011.5,NaN,NaN
2012-06-30 08:00:00,10.2,3.5,63.0,NaN,NaN,170.0,5.4,NaN,1013.1,NaN,NaN
2012-06-30 11:00:00,6.7,4.4,85.0,NaN,NaN,360.0,3.6,NaN,1015.0,NaN,NaN
2012-06-30 14:00:00,5.1,4.5,96.0,NaN,NaN,NaN,0.0,NaN,1015.2,NaN,NaN


In [5]:
df

,temp,dwpt,rhum,wdir,wspd,pres
time,,,,,,
2012-06-30 02:00:00,16.0,6.7,54.0,350.0,7.6,1012.6
2012-06-30 05:00:00,16.8,4.1,43.0,220.0,11.2,1011.5
2012-06-30 08:00:00,10.2,3.5,63.0,170.0,5.4,1013.1
2012-06-30 11:00:00,6.7,4.4,85.0,360.0,3.6,1015.0
2012-06-30 14:00:00,5.1,4.5,96.0,NaN,0.0,1015.2
...,...,...,...,...,...,...
2013-07-01 11:00:00,7.9,7.1,95.0,140.0,7.6,1023.8
2013-07-01 14:00:00,5.6,5.5,99.0,NaN,0.0,1023.5
2013-07-01 17:00:00,3.4,3.3,99.0,NaN,0.0,1023.3


In [4]:
# Drop the unnecessary columns

df = df.drop(['prcp', 'snow', 'wpgt', 'tsun', 'coco'], axis=1)
#Describe 
display(df.describe().transpose(),df.head())

,count,mean,std,min,25%,50%,75%,max
temp,2917.0,16.204628,6.952093,-1.3,11.4,16.4,20.8,42.8
dwpt,2915.0,9.893139,5.469332,-4.7,5.4,10.1,14.3,22.0
rhum,2915.0,70.605489,21.701572,11.0,53.5,73.0,91.0,99.0
wdir,2106.0,177.264957,94.569022,10.0,100.0,180.0,240.0,360.0
wspd,2931.0,8.475981,7.909435,0.0,0.0,7.6,13.0,46.4
pres,2931.0,1017.464722,6.365418,992.1,1013.0,1017.4,1022.0,1033.9


,temp,dwpt,rhum,wdir,wspd,pres
time,,,,,,
2012-06-30 02:00:00,16.0,6.7,54.0,350.0,7.6,1012.6
2012-06-30 05:00:00,16.8,4.1,43.0,220.0,11.2,1011.5
2012-06-30 08:00:00,10.2,3.5,63.0,170.0,5.4,1013.1
2012-06-30 11:00:00,6.7,4.4,85.0,360.0,3.6,1015.0
2012-06-30 14:00:00,5.1,4.5,96.0,NaN,0.0,1015.2


In [10]:
#Save the dataframe
#df.to_csv('../2012-2013 Weather data meteostat.csv')